In [6]:
import pandas as pd
import fnmatch
import dask.dataframe as dd
from intake.source.utils import reverse_format
import os
import re
import subprocess
from tqdm.auto import tqdm
from pathlib import Path
import shutil
import numpy as np


## Create text file containing all files available

In [7]:
def get_file_list(persist_path):
    persist_path = Path(persist_path)
    if persist_path.exists():
        shutil.rmtree(persist_path)
    persist_path.mkdir()
    root = Path("/glade/collections/cmip/CMIP6")
    dirs = [x for x in root.iterdir() if x.is_dir()]
    for directory in tqdm(dirs):
        print(directory)
        stem = directory.stem
        f = open(f"{persist_path}/{stem}.txt", "w")
        cmd = ["find", "-L", directory.as_posix(), "-name", "*.nc"]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=f)
        p.wait()
        

In [8]:
persist_path = "./CMIP6_filelist"
get_file_list(persist_path)

/glade/collections/cmip/CMIP6/PMIP
/glade/collections/cmip/CMIP6/HighResMIP
/glade/collections/cmip/CMIP6/ScenarioMIP
/glade/collections/cmip/CMIP6/AerChemMIP
/glade/collections/cmip/CMIP6/RFMIP
/glade/collections/cmip/CMIP6/OMIP
/glade/collections/cmip/CMIP6/C4MIP
/glade/collections/cmip/CMIP6/GeoMIP
/glade/collections/cmip/CMIP6/DCPP
/glade/collections/cmip/CMIP6/CMIP
/glade/collections/cmip/CMIP6/CFMIP
/glade/collections/cmip/CMIP6/LUMIP
/glade/collections/cmip/CMIP6/GMMIP
/glade/collections/cmip/CMIP6/PAMIP
/glade/collections/cmip/CMIP6/DAMIP
/glade/collections/cmip/CMIP6/LS3MIP
/glade/collections/cmip/CMIP6/CDRMIP




## Extract attributes of a file using information from CMI6 DRS.


References
 1. CMIP6 DRS: http://goo.gl/v1drZl
 2. Controlled Vocabularies (CVs) for use in CMIP6:
    https://github.com/WCRP-CMIP/CMIP6_CVs
    
    
Directory structure =
```<mip_era>/
    <activity_id>/
        <institution_id>/
            <source_id>/
                <experiment_id>/
                    <member_id>/
                        <table_id>/
                            <variable_id>/
                                <grid_label>/
                                    <version>
```
file name =
```<variable_id>_<table_id>_<source_id>_<experiment_id >_<member_id>_<grid_label>[_<time_range>].nc```
For time-invariant fields, the last segment (time_range) above is omitted.
Example when there is no sub-experiment: `tas_Amon_GFDL-CM4_historical_r1i1p1f1_gn_196001-199912.nc`
Example with a sub-experiment:   `pr_day_CNRM-CM6-1_dcppA-hindcast_s1960-r2i1p1f1_gn_198001-198412.nc`


In [9]:
activity_ids = list(Path(persist_path).rglob("*.txt"))
activity_ids = [activity_id.stem for activity_id in activity_ids]
activity_ids

['CFMIP',
 'CMIP',
 'LUMIP',
 'C4MIP',
 'LS3MIP',
 'RFMIP',
 'OMIP',
 'HighResMIP',
 'DCPP',
 'CDRMIP',
 'AerChemMIP',
 'GeoMIP',
 'PMIP',
 'DAMIP',
 'PAMIP',
 'GMMIP',
 'ScenarioMIP']

In [10]:
df = dd.read_csv(f"{persist_path}/*.txt", header=None).compute()
df.columns = ["path"]
df.head()

,path
0,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...



- 10/13/2019: 1,027,617 
- 10/15/2019: 1,113,227
- 10/16/2019: 1,129,214
- 10/16/2019: 1,138,743
- 10/17/2019: 2,352,702
- 10/19/2019: 2,492,108

In [11]:
len(df)

2492108

In [12]:
def _reverse_filename_format(file_basename, filename_template=None, gridspec_template=None):
    """
    Uses intake's ``reverse_format`` utility to reverse the string method format.
    Given format_string and resolved_string, find arguments
    that would give format_string.format(arguments) == resolved_string
    """
    try:
        return reverse_format(filename_template, file_basename)
    except ValueError:
        try:
            return reverse_format(gridspec_template, file_basename)
        except:
            print(
                f'Failed to parse file: {file_basename} using patterns: {filename_template} and {gridspec_template}'
            )
            return {}
            
def _extract_attr_with_regex(input_str, regex, strip_chars=None):
    pattern = re.compile(regex, re.IGNORECASE)
    match = re.findall(pattern, input_str)
    if match:
        match = max(match, key=len)
        if strip_chars:
            match = match.strip(strip_chars)

        else:
            match = match.strip()

        return match

    else:
        return None
    

exclude_patterns = ['*/files/*', '*/latest/*']
def _filter_func(path):
    return not any(
        fnmatch.fnmatch(path, pat=exclude_pattern) for exclude_pattern in exclude_patterns
    )



In [13]:
%%time
files = df.path.tolist()
filelist = list(filter(_filter_func, files))

CPU times: user 8.34 s, sys: 10.3 ms, total: 8.35 s
Wall time: 8.35 s


In [14]:
len(filelist)

1205130

In [15]:
def get_attrs(filepath):
    basename = os.path.basename(filepath)
    dirname = os.path.dirname(filepath)
    filename_template = '{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_label}_{time_range}.nc'

    gridspec_template = (
                '{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_label}.nc'
            )
    
    f = _reverse_filename_format(
            basename, filename_template=filename_template, gridspec_template=gridspec_template
        )

    fileparts = {}
    fileparts.update(f)
    parent = os.path.dirname(filepath).strip('/')
    parent_split = parent.split(f"/{fileparts['source_id']}/")
    part_1 = parent_split[0].strip('/').split('/')
    grid_label = parent.split(f"/{fileparts['variable_id']}/")[1].strip('/').split('/')[0]
    fileparts['grid_label'] = grid_label
    fileparts['activity_id'] = part_1[-2]
    fileparts['institution_id'] = part_1[-1]
    version_regex = r'v\d{4}\d{2}\d{2}|v\d{1}'
    version = _extract_attr_with_regex(parent, regex=version_regex) or 'v0'
    fileparts['version'] = version
    fileparts['path'] = filepath
    return fileparts 

In [16]:
%%time
entries = list(map(get_attrs, filelist))

CPU times: user 28.4 s, sys: 1.06 s, total: 29.5 s
Wall time: 29.5 s


In [17]:
entries[0]

{'variable_id': 'pr',
 'table_id': 'day',
 'source_id': 'BCC-ESM1',
 'experiment_id': 'ssp370',
 'member_id': 'r2i1p1f1',
 'grid_label': 'gn',
 'time_range': '20150101-20551231',
 'activity_id': 'AerChemMIP',
 'institution_id': 'BCC',
 'version': 'v20190702',
 'path': '/glade/collections/cmip/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp370/r2i1p1f1/day/pr/gn/v20190702/pr/pr_day_BCC-ESM1_ssp370_r2i1p1f1_gn_20150101-20551231.nc'}

In [18]:
# a = '/glade/collections/cmip/CMIP6/GMMIP/NCAR/NCAR/CESM2/amip-hist/r2i1p1f1/day/prc/gn/v20190814/prc_day_CESM2_amip-hist_r2i1p1f1_gn_19900101-19991231.nc'
# b = '/glade/collections/cmip/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp370/r2i1p1f1/day/pr/gn/v20190702/pr/pr_day_BCC-ESM1_ssp370_r2i1p1f1_gn_20150101-20551231.nc'
# c = '/glade/collections/cmip/CMIP6/GMMIP/NCAR/NCAR/CESM2/amip-hist/r2i1p1f1/day/clt/gn/v20190814/clt_day_CESM2_amip-hist_r2i1p1f1_gn_19200101-19291231.nc',

In [19]:
len(entries)

1205130

In [20]:
df1 = pd.DataFrame(entries)
df1.head()

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path
0,pr,day,BCC-ESM1,ssp370,r2i1p1f1,gn,20150101-20551231,AerChemMIP,BCC,v20190702,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,vmo,Omon,BCC-ESM1,ssp370,r2i1p1f1,gn,205501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,hfls,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,prsn,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,va,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


In [21]:
len(df1)

1205130

In [22]:
# Some entries are invalid
invalids = df1[~df1.activity_id.isin(activity_ids)]
df = df1[df1.activity_id.isin(activity_ids)]
invalids

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path
609133,sftlf,fx,abrupt-4xCO2,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
609134,areacella,fx,abrupt-4xCO2,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
610743,sftlf,fx,historical,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
612939,areacella,fx,piControl,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
614044,areacella,fx,1pctCO2,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
623072,sftlf,fx,hist-GHG,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/DAMIP/NCC/NorESM...


In [23]:
invalids.path.tolist()

## Keep latest version

['/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/abrupt-4xCO2/r1i1p1f1/fx/sftlf/gn/v20190815/sftlf/sftlf_fx_abrupt-4xCO2_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/abrupt-4xCO2/r1i1p1f1/fx/areacella/gn/v20190815/areacella/areacella_fx_abrupt-4xCO2_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/historical/r1i1p1f1/fx/sftlf/gn/v20190815/sftlf/sftlf_fx_historical_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/piControl/r1i1p1f1/fx/areacella/gn/v20190815/areacella/areacella_fx_piControl_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/1pctCO2/r1i1p1f1/fx/areacella/gn/v20190815/areacella/areacella_fx_1pctCO2_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/DAMIP/NCC/NorESM2-LM/hist-GHG/r1i1p1f1/fx/sftlf/gn/v20190815/sftlf/sftlf_fx_hist-GHG_NorESM2-LM_r1i1p1f1_gn.nc']

## Pick the latest versions

In [24]:
grpby = list(set(df.columns.tolist()) - {'path', 'version'})
groups = df.groupby(grpby)

In [25]:
%%time
idx_to_remove = []
for _, group in groups:
    if group.version.nunique() > 1:
        idx_to_remove.extend(group.sort_values(by=['version'], ascending=False).index[1:].values.tolist())

CPU times: user 5min 47s, sys: 6.51 s, total: 5min 53s
Wall time: 5min 50s


In [26]:
len(idx_to_remove)

23973

In [27]:
len(df)

1205124

In [28]:
df = df.drop(index=idx_to_remove)
len(df)

1181151

In [29]:
df["dcpp_init_year"] = df.member_id.map(lambda x: float(x.split("-")[0][1:] if x.startswith("s") else np.nan))
df["member_id"] = df["member_id"].map(lambda x: x.split("-")[-1] if x.startswith("s") else x)
df.head()

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path,dcpp_init_year
0,pr,day,BCC-ESM1,ssp370,r2i1p1f1,gn,20150101-20551231,AerChemMIP,BCC,v20190702,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN
1,vmo,Omon,BCC-ESM1,ssp370,r2i1p1f1,gn,205501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN
2,hfls,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN
3,prsn,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN
4,va,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN


In [30]:
columns = ["activity_id", "institution_id", "source_id", "experiment_id", "member_id", "table_id", "variable_id",
           "grid_label", "dcpp_init_year", "version", "time_range", "path"]
df = df[columns]
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
0,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,day,pr,gn,NaN,v20190702,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Omon,vmo,gn,NaN,v20190624,205501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,hfls,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,prsn,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,va,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


In [36]:
df.to_csv("../catalogs/glade-cmip6.csv.gz", compression="gzip", index=False)

In [37]:
len(df)

1181151